In [ ]:
from numpy import dot
from numpy.linalg import norm
import statistics
import csv
import copy

In [ ]:
import numpy as np
from numpy.linalg import norm
from numpy import dot

class kmeans:

    def __init__(self, n_clusters, max_iter=100):
        self.n_clusters = n_clusters
        self.max_iter = max_iter

        
    def jaccard_similarity(self,a,b):
        a=set(a)
        b=set(b)
        s1=a.intersection(b)
        s2=a.union(b)
        return len(s1)/len(s2)
    
    def cosine_similarity(self,v1,v2):
        return dot(v1, v2)/(norm(v1)*norm(v2))

    def initializ_centroids(self, X): # other initialization
        random_idx = np.random.permutation(X.shape[0])
        centroids = X[random_idx[:self.n_clusters]]
        return centroids

    def compute_centroids(self, X, labels):
        centroids = np.zeros((self.n_clusters, X.shape[1]))
        for k in range(self.n_clusters):
            centroids[k, :] = np.mean(X[labels == k, :], axis=0)
        return centroids
    
    def metadata(self,dfc,labels):
        cluster_num = self.n_clusters
        y= dfc[:,0]
        X = dfc[:,1:]  #preparing values in numpy format
        sum = np.zeros((cluster_num, X.shape[1]))
        sum_sq = np.zeros((cluster_num, X.shape[1]))
        N = np.zeros(cluster_num)
        #new_centroids = np.zeros((cluster_num, X.shape[1]))     #new centroids
        movieid = []
        for k in range(cluster_num):
            sum[k,:] = np.sum(X[labels==k,:],axis=0)
            sum_sq[k,:]=  np.sum((X[labels==k,:])**2,axis=0) 
            N[k] = X[labels==k].shape[0] 
            #new_centroids[k] = sum[k]/N[k]
            movieid.append(list(y[labels==k]))
        return (N,sum,sum_sq,movieid)
    


    def compute_distance(self, X, centroids):
        distance = np.zeros((X.shape[0], self.n_clusters))
        for k in range(self.n_clusters):
            for j in range(X.shape[0]):
                distance[j][k]=self.cosine_similarity(X[j],centroids[k])
        #pring(distance.shape)
        return distance


    
    def centroids(self, X):
        self.centroids = self.initializ_centroids(X[:,1:])
        for i in range(self.max_iter):  #it would summarise any x input giveen to it
            old_centroids = self.centroids
            distance = self.compute_distance(X[:,1:], old_centroids)
            self.labels = np.argmax(distance,axis=1) # finding closest distance returns an array of row indices for which 
            self.centroids = self.compute_centroids(X[:,1:], self.labels) 
            if np.all(old_centroids == self.centroids):
                #print(self.centroids)
                return self.metadata(X,self.labels)
        return self.metadata(X,self.labels) 


In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd



class bfr:

    def __init__(self, n_clusters,meta, max_iter=100):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.meta = meta
        self.centroids = self.compute_centroids(self.meta,self.n_clusters)

    
    def jaccard_similarity(self,a,b):
        a=set(a)
        b=set(b)
        s1=a.intersection(b)
        s2=a.union(b)
        return len(s1)/len(s2)
    
    def cosine_similarity(self,v1,v2):
        denom = norm(v1)*norm(v2)
        if denom == 0 :
            denom = 0.00000001
        return dot(v1, v2)/denom
    
    def compute_centroids(self, summary,cluster_num):  #takes in summed summary
        centroids = np.zeros((cluster_num, len(summary[1][1])))
        for k in range(cluster_num):
            centroids[k, :] = summary[1][k]/summary[0][k]
        return centroids
    

    
    def metadata(self,dfc,labels,cluster_num):
        y= dfc[:,0]
        X=dfc[:,1:]
        sum = np.zeros((cluster_num, X.shape[1]))
        sum_sq = np.zeros((cluster_num, X.shape[1]))
        N = np.zeros(cluster_num)
        #new_centroids = np.zeros((cluster_num, X.shape[1]))     #new centroids
        movieid = []
        for k in range(cluster_num):
            sum[k,:] = np.sum(X[labels==k,:],axis=0)   + self.meta[1][k]
            sum_sq[k,:]=  np.sum((X[labels==k,:])**2,axis=0) + self.meta[2][k]
            N[k] = X[labels==k].shape[0] + self.meta[0][k]
            #new_centroids[k] = sum[k]/N[k]
            movieid.append(list(y[labels==k]))
            #movieid[k].extend(self.meta[3][k])
        return (N,sum,sum_sq,movieid)
    


    def compute_distance(self, X, centroids,cluster_num):
        distance = np.zeros((X.shape[0], cluster_num))
        #print(len(centroids))
        for k in range(cluster_num):
            for j in range(X.shape[0]):
                distance[j][k]=self.cosine_similarity(X[j],centroids[k])  #centroid is the k column
        #pring(distance.shape)
        return distance


    
    def summarize(self, X):  #expects a centroid input
        distance = self.compute_distance(X[:,1:], self.centroids,self.n_clusters)
        self.labels = np.argmax(distance,axis=1) # finding closest distance returns an array of row indices for which 
        self.lab_dist = np.max(distance,axis=1)
        return self.metadata(X,self.labels,self.n_clusters) # take in dfd here and mordify metadata, dfd has all required information
            
    def active_centroid(self):
        return self.centroids
  

In [ ]:
def clustering(n):
    file = open('item_based.csv','r',buffering=100000,encoding='utf-8')
    chunk_num=0
    results = ''
    f=[]
    wr=[]
    for i in range(n):
        f.append(open('cluster'+str(i)+'.csv','a',newline='',encoding='utf-8'))
        wr.append(csv.writer(f[i], delimiter=','))
    while True:
        c=0
        tt=[]
        for line in file:
            tt.append(line.replace('\n','').split(','))
            c += 1
            if c==200:
                break 
        tt=np.array(tt)
        tt1=copy.deepcopy(tt)                                  #added
        tt1[np.where(tt1=='')]=np.nan
        tx=pd.DataFrame(np.array(tt1[:,1:],dtype='f'))
        #print(tx.head(20))
        tx=np.array(tx.apply(lambda x: x-np.mean(x),axis=1).fillna(0))
        #print('GOT HERE')
        #tx=np.array(tt1[:,1:],dtype='f')
        ind=np.arange(0,tt.shape[0])
        ind =ind.reshape([-1,1])
        ty=np.concatenate((ind,tx),axis=1)
        if chunk_num ==0 :
            kr=kmeans(n)
            results=kr.centroids(ty)
            chunk_num += 1
        else:
            ck = bfr(n,results)
            results = ck.summarize(ty) 
        for i in range(len(results[3])):
            for j in results[3][i]:
                #results1[i].append(tt[int(j),:])
                wr[i].writerow(tt[int(j),:])
        if c<200:
            break
    file.close()
    for i in range(n):
        f[i].close()
    return 

In [ ]:
def main():
    n=input('Please Enter the number of clusters k :')
    print('Please wait\nGenerating Clusters........')
    clustering(int(n))
    print('Done...Please check output files')
    return
main()